In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import csv

def connect_server(username, password): #Function to Create Connection with the SMTP server of Office365
    mailserver =  smtplib.SMTP('smtp.office365.com', 587)
    mailserver.ehlo()
    mailserver.starttls()
    mailserver.login(username, password)
    return mailserver

def disconnect_server(msg, mailserver):
    del msg
    mailserver.quit()

def setup_message(username, to, subject, body): #Setting the Email with Sender , Receiver and Subject Details
    msg = MIMEMultipart()
    msg['From'] = username
    msg['To'] = to
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    return msg

def process_email(details_filename, username, password, subject):
    mailserver = connect_server(username, password)

    with open(details_filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        with open('HRMail.txt', 'r', encoding='utf-8') as file:  # reading the Content of the Email from a text file and setting the body of the email
            body = file.read()
        for row in csv_reader: # Loop created to send the Email to the entire list in the CSV file
            try:
                to = row[0]
                msg = setup_message(username, to, subject, body)
                p = MIMEBase('application', 'octet-stream')
                q = MIMEBase('application','octet-stream')
                attachment = open("ABC.pdf", "rb") # Reading a file in binary format
                attach = open("1.png","rb")
                p.set_payload(attachment.read()) # Setting up the Read file as payload to be attached to the email as an attachment
                q.set_payload(attach.read())
                encoders.encode_base64(p) # Encoding the attachments to maintain file integrity
                encoders.encode_base64(q)
                p.add_header('Content-Disposition', "attachment; filename=filename.pdf")
                q.add_header('Content-Disposition','attachment; filename=filename1.png')
                msg.attach(p) # Attaching the file to the Email after proper encoding
                msg.attach(q)
                mailserver.send_message(msg) # Sending the Email
            except Exception as e:
                print(e , "Exception")
    disconnect_server(msg, mailserver)

username = input("Email ID")
password = input("Enter Password")
process_email("CSV File with Emails", username, password, "Subject of Email")